#### Chapter 8 - Integer Programming The Transportation Algorithm 

8.11 A regional airline can buy its jet fuelt from any one of three vendors. The airline's needs for the upcoming month at each of the three airports it serves are 100,000 gal at airport 1, 180,000 gal at airport 2, and 350,000 gal at airport 3. Each vendor can supply fuel to each airport at a price (in cents per gallon) given by the following schedule. 

Airport 1, Airport 2, Airport 3, 
Vendor 1, 92, 89, 90, 
vendor 2, 91, 91, 95, 
vendor 3, 87, 90, 92 

Each vendor is limited in the total number of gallons it can provide during any one month. These capacities are 320,000 gal for vendor 1, 270,000 gal for vendor 2 and 190,000 gal for vendor 3. Determine a purchasing policy that will supply the airlines's requirements at each airport at minimum total cost. 

In [5]:
from pulp import *

# Define airports and vendors
airports = ["Airport 1", "Airport 2", "Airport 3"]
vendors = ["Vendor 1", "Vendor 2", "Vendor 3"]

# Define fuel needs for each airport
demand = {"Airport 1": 100000, "Airport 2": 180000, "Airport 3": 350000}

# Define fuel prices from each vendor to each airport
prices = {
    "Vendor 1": {"Airport 1": 92, "Airport 2": 89, "Airport 3": 90},
    "Vendor 2": {"Airport 1": 91, "Airport 2": 91, "Airport 3": 95},
    "Vendor 3": {"Airport 1": 87, "Airport 2": 90, "Airport 3": 92}
}

# Define vendor capacities
capacities = {"Vendor 1": 320000, "Vendor 2": 270000, "Vendor 3": 190000}

# Create a linear programming model
model = LpProblem("Minimize Fuel Cost", LpMinimize)

# Define variables for the amount of fuel purchased from each vendor to each airport
fuel_purchase = LpVariable.dicts("Fuel_Purchase", [(v, a) for v in vendors for a in airports], lowBound=0)

# Set objective function: minimize total fuel cost
model += lpSum([prices[v][a] * fuel_purchase[(v, a)] for v in vendors for a in airports]), "Total Fuel Cost"

# Add constraints:
# 1. Meet demand at each airport
for a in airports:
    model += lpSum([fuel_purchase[(v, a)] for v in vendors]) == demand[a], f"Demand at {a}"

# 2. Respect vendor capacity
for v in vendors:
    model += lpSum([fuel_purchase[(v, a)] for a in airports]) <= capacities[v], f"Capacity of {v}"

# Solve the model
model.solve()

# Print the status and solution
print("Status:", LpStatus[model.status])
print("Optimal Solution:")
for v in vendors:
    for a in airports:
        if fuel_purchase[(v, a)].varValue > 0:
            print(f"Buy {fuel_purchase[(v, a)].varValue} gallons from {v} to {a}")

# Calculate and print total cost
total_cost = value(model.objective)
print(f"Total Fuel Cost: ${total_cost / 100:.2f}")

Status: Optimal
Optimal Solution:
Buy 320000.0 gallons from Vendor 1 to Airport 3
Buy 120000.0 gallons from Vendor 2 to Airport 2
Buy 100000.0 gallons from Vendor 3 to Airport 1
Buy 60000.0 gallons from Vendor 3 to Airport 2
Buy 30000.0 gallons from Vendor 3 to Airport 3
Total Fuel Cost: $565800.00


8.13 Two drug companies have inventories of 1.1 and .9 million doses of a particular flu vaccine and an epidemic of the flu seems imminent in three cities. Since the flu could be fatal to senior citizens it is imperative they be vaccinated first; others will be vaccinated on a first-come first served basis while the vaccine supply lasts. The amounts of vaccine (in millions of doses) each city estimates it could administer are as follows: 
City 1, City 2, City 3, 
To elders, .325, .260, .195, 
to others, .750, .8, .650 

The shipping costs (in cents per dose) between drug companies and cities are as follows: 

City 1, City 2, City 3, 
Company 1, 3, 3, 6, 
Company 2, 1, 4, 7 

Determine a minimum-cost shipping schedule which will provide each city with at least enough vaccine to care for its senior citizens. 

In [6]:
from pulp import *

# Define cities and companies
cities = ["City 1", "City 2", "City 3"]
companies = ["Company 1", "Company 2"]

# Define vaccine needs for each city (in millions of doses)
demand_elders = {"City 1": 0.325, "City 2": 0.260, "City 3": 0.195}
demand_others = {"City 1": 0.750, "City 2": 0.800, "City 3": 0.650}

# Define shipping costs (in cents per dose)
shipping_costs = {
    "Company 1": {"City 1": 3, "City 2": 3, "City 3": 6},
    "Company 2": {"City 1": 1, "City 2": 4, "City 3": 7}
}

# Define company inventories (in millions of doses)
inventories = {"Company 1": 1.1, "Company 2": 0.9}

# Create a linear programming model
model = LpProblem("Minimize Shipping Cost", LpMinimize)

# Define variables for the amount of vaccine shipped from each company to each city
shipment = LpVariable.dicts("Shipment", [(c, ci) for c in companies for ci in cities], lowBound=0)

# Set objective function: minimize total shipping cost
model += lpSum([shipping_costs[c][ci] * shipment[(c, ci)] for c in companies for ci in cities]), "Total Shipping Cost"

# Add constraints:
# 1. Meet elder demand at each city
for ci in cities:
    model += lpSum([shipment[(c, ci)] for c in companies]) >= demand_elders[ci], f"Elder Demand at {ci}"

# 2. Respect company inventory
for c in companies:
    model += lpSum([shipment[(c, ci)] for ci in cities]) <= inventories[c], f"Inventory of {c}"

# Solve the model
model.solve()

# Print the status and solution
print("Status:", LpStatus[model.status])
print("Optimal Solution:")
for c in companies:
    for ci in cities:
        if shipment[(c, ci)].varValue > 0:
            print(f"Ship {shipment[(c, ci)].varValue:.3f} million doses from {c} to {ci}")

# Calculate and print total cost
total_cost = value(model.objective)
print(f"Total Shipping Cost: ${total_cost / 100:.2f} million")

Status: Optimal
Optimal Solution:
Ship 0.260 million doses from Company 1 to City 2
Ship 0.195 million doses from Company 1 to City 3
Ship 0.325 million doses from Company 2 to City 1
Total Shipping Cost: $0.02 million
